In [2]:
import json
import sys
import binascii
import base64
from pydub import AudioSegment
import soundfile as sf

In [54]:
def convert_m4a_to_flac(m4a_path, flac_path):
    # Load m4a file
    audio = AudioSegment.from_file(m4a_path, format="m4a")

    # Convert stereo to mono
    audio = audio.set_channels(1)

    # Convert to wav
    audio.export("temp.wav", format="wav")

    # Load wav file
    data, samplerate = sf.read('temp.wav')

    # Save as FLAC
    sf.write(flac_path, data, samplerate, format='flac')

# Example usage
convert_m4a_to_flac("benoit1.m4a", "benoit1.flac")


In [55]:
convert_m4a_to_flac("benoit2.m4a", "benoit2.flac")
convert_m4a_to_flac("caitlin.m4a", "caitlin.flac")


In [28]:
from web3 import Web3

w3 = Web3(Web3.HTTPProvider('http://127.0.0.1:8545', request_kwargs={'timeout': 6000}))


In [30]:
with open('/Users/benoitrobaglia/Documents/eth_paris/rollups-examples/deployments/localhost/InputFacet.json', 'rt') as fin:
    input_facet = json.load(fin)

In [14]:
#acct1 = w3.eth.accounts[0]


In [31]:
acct2 = w3.eth.account.from_key("0xac0974bec39a17e36ba4a6b4d238ff944bacb478cbed5efcae784d7bf4f2ff80")

In [32]:
some_address = "0xF8C694fd58360De278d5fF2276B7130Bfdc0192A"


In [33]:
cartesi_input = w3.eth.contract(address=some_address, abi=input_facet['abi'])


In [35]:
def load_and_convert_flac(flac_path, function=0):
    # Open file and read as bytes
    with open(flac_path, 'rb') as wf:
        wfr = wf.read()

    # Base64 encode the bytes
    audio_b64bytes = base64.b64encode(wfr)
    # Decode it to string for putting into JSON
    audio_b64str = audio_b64bytes.decode()

    # Create JSON string
    json_string = json.dumps({"function": function, "audio_bytes": audio_b64str})
    # Convert the string to bytes
    json_bytes = json_string.encode()
    # Convert to hex
    return Web3.toHex(json_bytes)


In [56]:
ben1 = load_and_convert_flac('benoit1.flac', 0)
ben2 = load_and_convert_flac('benoit2.flac', 1)
cait = load_and_convert_flac('caitlin.flac', 1)

In [57]:
%%time
input_tx = cartesi_input.functions.addInput(ben2).build_transaction({
    "from": acct2.address,
    "nonce": w3.eth.get_transaction_count(acct2.address),
})

/Users/benoitrobaglia/opt/anaconda3/lib/python3.9/site-packages/web3/eth.py:656: UserWarning: There was an issue with the method eth_maxPriorityFeePerGas. Calculating using eth_feeHistory.
  warnings.warn(


CPU times: user 126 ms, sys: 50.4 ms, total: 177 ms
Wall time: 7min 16s


In [58]:
signed_tx = w3.eth.account.sign_transaction(input_tx, private_key=acct2.key)

In [59]:
w3.eth.send_raw_transaction(signed_tx.rawTransaction)

HexBytes('0xdc2d813269ca133a8974ee570d7db4c26e0818e01665e7152f7c1c192e0db16d')

In [60]:
w3.toBytes('0x707574207468652070756c7020696e746f2061206261736520616e6420776974682074776f206f756e636573206f66206d656c7465642062757474657220746f207461626c6573706f6f6e2066616c6c73206f66206c656d6f6e206a756963652068616c66206120706f756e64206f66206a757374206e75747320626f696c656420616e642067726174656420616e6420736561736f6e696e67206f662073616c7420616e642077686974652070657070657220746f207461737465')

TypeError: expected a bool, int, byte or bytearray in first arg, or keyword of hexstr or text